In [2]:
import re
alt_text = "IX.8.5 Pompeii. May 2006. Looking south to entrance doorway of shop. According to Della Corte, a recommendation found on the east (left) side between IX.8.6 and IX.8.5, associated a certain Urbanus with the proprietor Verus –Ur(banus rogat), idem facit   [CIL IV 3751]"

re.findall("[I][X,V,I]\.[1-9]\.[1-9]",alt_text)


['IX.8.5', 'IX.8.6', 'IX.8.5']

In [ ]:
addresses_dict = {}
number_of_rows_to_run = 70000
start_id = 1
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)

#Do image descriptions first
cursor.execute("SELECT id as wi_id, img_alt FROM webpage_images.tbl_webpage_images WHERE id>=%s AND id<%s", (start_id, end_id))
for wi_id, img_alt in cursor:
        
    #print(img_alt)
    addresses = (re.findall("[1IVX]+\.[1-9][\d]*\.[0-9a-z]+",img_alt))
    if (len(addresses)==0):
        #print(img_alt)
        addresses = (re.findall("[1IVX]+\.[1-9][\d]*\.*",img_alt))
        #if (len(addresses)==0):
            #print(img_alt)
    #else:
        #print(addresses)
    for cn in range(0,len(addresses)):
        addresses_dict[addresses[cn]] = addresses[cn]
        
mariadb_connection.close()

for a, a1 in sorted(addresses_dict.items()):    
    print (a)

In [ ]:
# We have a problem where both of these cases produce the same address I.1.-  
# However, it's useful to leave this for now to see what in the alt text is causing this. 
I.1
=I=1=-=
I.1.
=I=1=-=


def raw_address_parse(raw_address):
    # Raw addresses are formatted like: I.10.11
    regio=""
    insula=""
    entrance=""
    if (raw_address is None):
        return

    if (raw_address.strip() == ""):
        return
    
    # if there is no . assume there is no address
    if(raw_address.find(".")<0):
        return
    else:
        #print(tag_text)
        fp = raw_address.find(".")
        regio = raw_address[:fp]
        non_regio = raw_address[fp+1:]
        sp = non_regio.find(".")
        if(non_regio.find(".")<0):
            insula = non_regio
            entrance = ""
        else:
            insula = non_regio[:sp]
            entrance = non_regio[sp+1:]
        if(entrance.strip() == ""):
            entrance="-"
        if(regio=="1"):
            regio="I"
        if(int(insula)>30):
            regio=""
            insula=""
            entrance=""
    print("="+regio+"="+insula+"="+entrance+"=")
    return
for a, a1 in sorted(addresses_dict.items()):    
    print (a)
    raw_address_parse(a)

In [ ]:
# insert tags into table
def raw_address_parse_insert(raw_address, cursor_insert):
    # Raw addresses are formatted like: I.10.11
    regio=""
    insula=""
    entrance=""
    if (raw_address is None):
        return

    if (raw_address.strip() == ""):
        return
    
    # if there is no . assume there is no address
    if(raw_address.find(".")<0):
        return
    else:
        #print(tag_text)
        fp = raw_address.find(".")
        regio = raw_address[:fp]
        non_regio = raw_address[fp+1:]
        sp = non_regio.find(".")
        if(non_regio.find(".")<0):
            insula = non_regio
            entrance = ""
        else:
            insula = non_regio[:sp]
            entrance = non_regio[sp+1:]
        if(entrance.strip() == ""):
            entrance="-"
        if(regio=="1"):
            regio="I"
        if(int(insula)>30):
            regio=""
            insula=""
            entrance=""
    if(regio>"" and insula>""):
        cursor_insert.execute("INSERT INTO tbl_addresses (pinp_regio, pinp_insula, pinp_entrance) VALUES ('"+regio+"', '"+insula+"', '"+entrance+"')")
        mariadb_connection.commit()

    return


import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor_insert = mariadb_connection.cursor(buffered=True)

for a, a1 in sorted(addresses_dict.items()):    
    
    a=a.replace("'","''")
    a=a.replace("and","")
    print (a)
    raw_address_parse_insert(a, cursor_insert)
mariadb_connection.close()


In [46]:
mariadb_connection.close()


In [ ]:
# insert tags into table
def raw_address_parse_insert_into_table(raw_address, wi_id, cursor_add, cursor_add_x, cursor_insert_add_x):
    # Raw addresses are formatted like: I.10.11
    regio=""
    insula=""
    entrance=""
    if (raw_address is None):
        return

    if (raw_address.strip() == ""):
        return
    
    # if there is no . assume there is no address
    if(raw_address.find(".")<0):
        return
    else:
        #print(tag_text)
        fp = raw_address.find(".")
        regio = raw_address[:fp]
        non_regio = raw_address[fp+1:]
        sp = non_regio.find(".")
        if(non_regio.find(".")<0):
            insula = non_regio
            entrance = ""
        else:
            insula = non_regio[:sp]
            entrance = non_regio[sp+1:]
        if(entrance.strip() == ""):
            entrance="-"
        if(regio=="1"):
            regio="I"
        if(int(insula)>30):
            regio=""
            insula=""
            entrance=""
    if(regio>"" and insula>""):
        #cursor_insert.execute("INSERT INTO tbl_addresses (pinp_regio, pinp_insula, pinp_entrance) VALUES ('"+regio+"', '"+insula+"', '"+entrance+"')")
        #mariadb_connection.commit()
        #We have an address.  Get its ID and then insert it into the cross reference table tbl_addresses_x with the ID of the address

        cursor_add.execute("SELECT id as add_id FROM webpage_images.tbl_addresses WHERE pinp_regio='"+regio+"' AND pinp_insula='"+insula+"' AND pinp_entrance='"+entrance+"'")
        for add_id in cursor_add:
            #print(add_id[0])
            cursor_add_x.execute("SELECT count(*) as cn FROM webpage_images.tbl_addresses_x WHERE add_id=%s AND wi_id=%s",(add_id[0],wi_id))
            for cn in cursor_add_x:
                #print("count " + str(cn[0]))
                if(cn[0]==0):
                    cursor_insert_add_x.execute("INSERT INTO tbl_addresses_x (add_id,wi_id) VALUES (%s, %s)",(add_id[0],wi_id))
                    mariadb_connection.commit()
                #else:
                    #print(str(add_id[0])+" "+str(wi_id)+" are already in the table.")


    return

number_of_rows_to_run = 70000
start_id = 1
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_add = mariadb_connection.cursor(buffered=True)
cursor_add_x = mariadb_connection.cursor(buffered=True)
cursor_insert_add_x = mariadb_connection.cursor(buffered=True)

#Do image descriptions first
cursor.execute("SELECT id as wi_id, img_alt FROM webpage_images.tbl_webpage_images WHERE id>=%s AND id<%s", (start_id, end_id))
for wi_id, img_alt in cursor:
    
    print(wi_id)
    if(not img_alt is None):
        #print(img_alt)
        addresses = (re.findall("[1IVX]+\.[1-9][\d]*\.[0-9a-z]+",img_alt))
        if (len(addresses)==0):
            #print(img_alt)
            addresses = (re.findall("[1IVX]+\.[1-9][\d]*\.*",img_alt))
            #if (len(addresses)==0):
                #print(img_alt)
            #else:
            #print(addresses)
        for cn in range(0,len(addresses)):
            print(addresses[cn])
            raw_address_parse_insert_into_table(addresses[cn], wi_id, cursor_add, cursor_add_x, cursor_insert_add_x)
        
mariadb_connection.close()









